# SQL Introduction
<hr style="border:.5px solid gray"> </hr>


<center><img src="www/logo3.png"></center> 



**Author: James Triveri**    
**Date: 2021-05-19**    
<br>     




# Topics:
<hr style="border:.75px solid gray"> </hr>

1. **Overview of SQL**   
<br> 
2. **Why SQL is important**     
<br>  
3. **Common operations, expressions and queries**   
<br>  
4. **Introduction to JOINs**  
<br> 
    i. **Leading causes of failed JOIN operations and robustness measures**   
<br> 
5. **Challenging *That's how we've always done it...* and avoidance of anti-patterns**   
<br> 
6. **How can I get started?**        
<br>     






# Overview
<hr style="border:.75px solid gray"> </hr>

- SQL (Structured Query Language) is a declarative programming language designed for managing structured data 
within a relational database management system (RDBMS).   
<br> 
- ***Declarative*** means you tell the optimizer what you want to achieve, rather than how to achieve it.     
<br> 
- Comparison of imperative and declarative code excerpts:  


# Code Comparison
<hr style="border:.75px solid gray"> </hr>

**Imperative excerpt:**
```python
# Typical iteration scheme in Python.
input_vals = [3, 5, 7, 11, 13, 17, 19, 23, 29, 31,]
output_vals = []

for ii in input_vals:
    output_vals.append(ii + 3)
```

<br>

**Declarative excerpt:**

```sqlite
-- Typical SQL JOIN operation.
SELECT 
    TBL_A.POLICYID, TBL_B.PAID_LOSS
FROM TBL_A LEFT JOIN TBL_B 
    ON TBL_A.POLICYID = TBL_B.POLICYID;
```

    

# Overview
<hr style="border:.75px solid gray"> </hr>

* Popular RDBMS include:  
<br>  
    - Oracle (on-premises database market share ~45%)
    - MySQL
    - **SQL Server** (on-premises database market share ~25%)
    - PostgreSQL
    - IBM DB2 (on-premises database market share ~15%)
    - SQLite
    - Amazon RedShift (AWS data warehouse)
    - Google Big Query (GCP data warehouse)

    
* Although RDBMSs offer proprietary extensions, ANSI-compliant SQL is a subset supported by all vendors, which requires 
executing the following  commands in the same way:       
<br> 
    - **UPDATE, DELETE, SELECT, INSERT, WHERE**   
  

* At GuideOne, SQL Server is the only analyst facing RDBMS, so cross-platform compatibility isn't a major concern. 



# Why bother learning SQL?
<hr style="border:.75px solid gray"> </hr>


- In a CNBC article titled ***5 skills that will lead to a high-paying in-demand job***, 
SQL was ranked 4th, ahead of Java (no. 5), and immediately following bilingualism (no. 3):   
<br>   
    * https://www.cnbc.com/2017/04/26/5-skills-that-will-lead-to-a-high-paying-in-demand-job.html  
<br> 
- Every popular programming language exposes a database API/interface. Whether working from Python, R or C++, 
properly written SQL can be executed from any client-side application without modification (more on 
"properly written" later).       
<br>
- Knowledge/familiarity of/with SQL is quickly surpassing Excel as the baseline core competency 
for technically-oriented roles:     
<br> 
    - Scalability 
    - SQL-related questions asked more frequently in technical interviews.   
<br>  
- Instills tenets of computational thinking (logic, assessment, patterns, automation, and generalisation).


# How much SQL do I need to learn?
<hr style="border:.75px solid gray"> </hr>

- Typically only a small subset of SQL commands are required for non-admin (e.g. analyst) work.  
<br> 
- *I know enough SQL to get what I need into R...* - **A. Niebrugge**     
<br>
- Even a little knowledge of SQL and data organization best practices can have a positive
influence on all aspects of the analytics pipeline.    
<br>   
- There is no compression algorithm for experience: Consistently use it, and practice, practice, practice!
<br>


# Data Used for Examples and Exercises
<hr style="border:.75px solid gray"> </hr>

* The queries used to demonstrate concepts target the *French Motor Third-Part Liability* 
database, a popular publicly available dataset used within the Actuarial modeling community.  
<br>
* Consists of risk features for motor third-part liability policies observed mostly on one year.
Data is partitioned into three tables (available in User_ActuarialPilot):  
<br> 
    - **SAMPLE_FREQUENCY** (413,169 rows): Contains the number of claims, exposures and additional 
    risk characteristics associated with each policy.  
    <br>   
    - **SAMPLE_SEVERITY** (16,180 rows): Contains policy identifier and paid loss amounts. policy id may
    be repeated for policies with multiple claims.   
    <br>
    - **SAMPLE_POPULATION** (10 rows): Mapping of French geographic region (roughly akin to a state) to 
    population and average latitude/longitude.  


# SQL Order of Operations
<hr style="border:.75px solid gray"> </hr>

Conventional SQL queries consist of a few common elements:

- **`SELECT`**: Specifies the columns to retrieve, along with any aliases. `*` is a wildcard, 
which returns all columns from the target table.         
<br>    
- **`FROM`**: Specifies the table from which the columns are to be retrieved.    
<br> 
- **`WHERE`**: One or more conditions used to filter the result set.    
<br>   
- **`GROUP BY`**: Frequently used in conjunction with aggregate operations 
(`SUM`, `MIN`, `MAX`, `AVG`, etc.). Represents the keys over which aggregates are to be computed.    
<br>    
- **`ORDER BY`**: How the result set should be sorted. `ORDER BY` is followed by a comma delimited
list of fieldnames along with `ASC/DESC` for ascending/descending. 


# Sample Query \#1
<hr style="border:.75px solid gray"> </hr>

Retrieve records from SAMPLE_FREQUENCY for policies with diesel engines having at least one claim. 
Reverse sort results by policy id:

<br>


```sqlite
SELECT * FROM SAMPLE_FREQUENCY
WHERE GAS = 'Diesel' AND NBR_CLAIMS > 1
ORDER BY POLICYID DESC;
```

<br>

- Execution in SSMS (sample query \#1)     




# SQL Order of Operations 
<hr style="border:.75px solid gray"> </hr>

Queries are executed in an order different than how they are written:  
<br>  


<center><img width="662" height="334" src="www/operations.png"></center> 




# Aggregate Operations 
<hr style="border:.75px solid gray"> </hr>

- **`GROUP BY`** enables the execution of aggregate operations over a specified set of one or more 
keys. Usually used in conjunction with aggregate functions:   
<br>   
    * **`AVG`**
    * **`COUNT`**
    * **`MAX`**
    * **`MIN`**
    * **`STDEV`**
    * **`STDEVP`**
    * **`SUM`**
    * **`VAR`**
    * **`VARP`**
    

# Sample Query \#2
<hr style="border:.75px solid gray"> </hr>

Compute the total number of claims, total number of exposures and average driver age by region
for cars strictly less than 5 years old. Sort results by region:    


```sqlite
SELECT
    REGION, 
    SUM(NBR_CLAIMS) NBR_CLAIMS, 
    SUM(EXPOSURE) EXPOSURE, 
    AVG(DRIVERAGE) DRIVERAGE
FROM SAMPLE_FREQUENCY WHERE CARAGE < 5
GROUP BY REGION 
    ORDER BY REGION ASC;
```

<br>

- Execution in SSMS (sample query \#2)      




# SQL JOINs 
<hr style="border:.75px solid gray"> </hr>

- JOINs are used to combine rows from two or more tables, based on one or more related columns 
between tables. Common JOINs include:  
<br> 
    * **`INNER JOIN`**
    * **`OUTER JOIN`**
    * **`FULL OUTER JOIN`**    
<br>    
- Refer to *SQL_JOINs_Reference.pdf* for additional information.   
<br>   
- Each JOIN will be presented from a set theoretic, a tabular and a SQL perspective.    
<br> 
- The examples that follow use only a single key for ease of demonstration, but JOINs between 
tables can be performed over any number of common columns, limited only by the number of columns
in the smaller of the two tables.   


# INNER JOIN: Set Theoretic Perspective
<hr style="border:.75px solid gray"> </hr>

An `INNER JOIN` is analogous to the intersection of two sets A and B:    
<br>
<center><img width="416" height="279" src="www/inner0.png"></center>



# INNER JOIN: Tabular Perspective
<hr style="border:.75px solid gray"> </hr>

Only those records with key values in both tables will be retained:  
<br>  

<center><img src="www/inner1.png"></center>



# INNER JOIN: SQL Perspective  
<hr style="border:.75px solid gray"> </hr>


```sqlite
SELECT
    A.POLICYID,
    A.REGION, 
    B.POP
FROM A INNER JOIN B 
    ON A.REGION = B.REGION
```
<br>  


- Execution in SSMS (sample query \#3)   
<br>
- It isn't required to prefix column names with their table of origin if
the column is defined in only one of the tables (like POLICYID or POP), 
but being explicit can improve readability, especially as queries grow 
in complexity. 




# OUTER JOIN: Set Theoretic Perspective
<hr style="border:.75px solid gray"> </hr>

An `OUTER JOIN` produces a complete set of records from Table A, with matching records 
(where available) in Table B. If there is no match, the right side (Table A values) will 
contain null:   
<br>
<center><img src="www/outer0.png"></center>



# OUTER JOIN: Tabular Perspective
<hr style="border:.75px solid gray"> </hr>

All records from Table A are retained. Key column matches from A will be populated, missing
values will be null:   
<br>

<center><img src="www/outer1.png"></center>



# OUTER JOIN: SQL Perspective
<hr style="border:.75px solid gray"> </hr>


```sqlite
SELECT
    A.POLICYID,
    A.REGION, 
    B.POP
FROM A LEFT OUTER JOIN B 
    ON A.REGION = B.REGION
```
<br> 
- Execution in SSMS (sample query \#4)       
<br> 
- `LEFT OUTER JOIN` is identical to `LEFT JOIN` (i.e., `OUTER` is frequently omitted).       
<br>
- The SQL Standard considers the first named table to be the left table, 
and the second named table to be the right table. In the query above, **A** is the left table
and **B** the right table.   
<br>
- If you want all the rows from the first table and any matching rows from the second table, 
use `LEFT JOIN / LEFT OUTER JOIN`. If you want all rows from the second table and any matching rows from 
the first table, use `RIGHT JOIN / RIGHT OUTER JOIN`.


# OUTER JOIN: Misc.
<hr style="border:.75px solid gray"> </hr>

- In Microsoft Excel, **VLOOKUP** is functionally equivalent to an outer join in SQL:     
<br>   

<center><img width="675" height="27" src="www/outer2.png"></center>

<br>

- ***lookup_value***: The key to lookup in the target table.   
<br>
- ***table_array***: The "right" table.   
<br>
- ***col_index_num***: Index offset of target column relative to key column.      
<br>
- ***range_lookup***: FALSE to specify exact key match, TRUE for partial key match.   




# FULL OUTER JOIN: Set Theoretic Perspective
<hr style="border:.75px solid gray"> </hr>

A `FULL OUTER JOIN` includes all rows from both tables. When no matching
values exist for rows in the left table, you'll see null values from the
result set on the right. When no matching values exist for rows on the
right table, you'll see null values from the result set on the left.
<br>

<center><img width="415" height="280" src="www/fouter0.png"></center>




# FULL OUTER JOIN: Tabular Perspective
<hr style="border:.75px solid gray"> </hr>

<br>   

<center><img width="645" height="197" src="www/fouter1.png"></center>



# FULL OUTER JOIN: SQL Perspective
<hr style="border:.75px solid gray"> </hr>

```sqlite
SELECT 
	A.POLICYID, 
	A.NBR_CLAIMS, 
	B.PAID_LOSS
FROM A FULL OUTER JOIN B
    ON A.POLICYID = B.POLICYID
```
<br>  
- Execution in SSMS (sample query \#5)       
<br> 
- `FULL OUTER JOIN` should be used with caution: Note that it is no longer possible to aggregate 
NBR_CLAIMS (or PAID_LOSS) in the result set to obtain an accurate claim count, since NBR_CLAIMS 
from table A is replicated for each key column match in table B. 

<br>




# Leading causes of failed JOINs
<hr style="border:.75px solid gray"> </hr>

- Failure of JOINs can be attributed to a number of factors. A few things to check:    
<br>  
    * If merging on a character column, did you trim whitespace (e.g., `LTRIM(RTRIM(<column_name>))`)?  
    <br> 
    * If merging on policy or claim numbers, ensure values are left-0-padded to the correct number of 
      digits (`21107` with not match `000021107`). Policy numbers should be 9 characters wide, 
      claim numbers 8 characters wide. This can be accomplished using:
      
          RIGHT('000000000' + POLICYNO, 9) POLICYNO
          RIGHT('00000000' + CLAIMNO, 8) CLAIMNO
           

# Challenging *That's how we've always done it...*
<hr style="border:.75px solid gray"> </hr>


- Four generations of cooks  
<br>   
- Proliferation of temp table usage in GuideOne queries     
<br>   
- Query used to compile historical policycenter exposures creates **77** temporary tables!?! 
This is undesirable for a number of reasons:  

     **i.** Logic to compile data not encapsulated in a single executable statement.    
     **ii.** Query is not portable / cannot be passed to client-side applications for direct execution.     
     **iii.** Temp tables are ephemeral, which makes debugging more difficult.      
<br>   
- *Temporary tables are fine for exploratory data analysis, but to the extent possible 
should not be used in queries intended for production or shared with others*.



# Organizing Complex Queries with SubQuery Factoring
<hr style="border:.75px solid gray"> </hr>

- Queries that rely on temporary tables can be converted into a single executable SQL statement using *subquery factoring*.   
<br>   
- As opposed to creating temporary tables, define each intermediate table at the
start of the query immediately following a `WITH` clause. Each subquery is assigned an alias,
which can then be referenced in the body of the main query.    
<br>   
- Example transforming a temp table query using subquery factoring in SSMS    



# How can I get started?
<hr style="border:.75px solid gray"> </hr>

1. Perform an inventory of the data acquisition routines that you are responsible for,
then answer the following:

    i. ***Does another individual provide the data for you?***     
    If yes, request that they share the SQL used to compile the dataset. Become 
    familiar with the most commonly used tables and fields, and how elements of each of 
    the databases are accessed.

    ii. ***Is the data acquisition routine already implemented in SQL?***     
    If yes, use subquery refactoring to replace instances of temporary table creation. 
    Ensure the transaction can be dispatched in a single executable statement. This 
    ensures portability across applications.   
<br>     
3. Learn how the organization manages data.    
<br>  
2. Be enthusiastic about the prospect of leveling-up your technical competency, and developing
a set of skills that will be highly in-demand for the rest of your career.      
<br>  
4. Again, there is no compression algorithm for experience: Try to use it a little bit every day.   
<br>   
5. Work through SQL Exercises.   
<br>  
